# Finetuning of BERT for NER

This notebook contains all the steps for the finetuning of French version of BERT, the objective is to extract specific information from french public business agreements. For this we have a dual approach, first we classify information and then only we use NER property of our model to extract wage variations.

Sources: This notebook is inspired from a masterclass and notebook provided by [Thomas Boehler](https://fr.linkedin.com/in/thomas-boehler-ba34a744) and also the work of my colleagues [Mouad Bernoussi](https://ma.linkedin.com/in/mouad-bernoussi-00aa91242) (see ./notebooks/external) and [Conrad Thiounn](https://github.com/cthiounn).

Try

1. Play with PyTorch
2. Play with the model 
3. Adapt data to BERT format (see Thomas Boehler)
4. Create training

In [11]:
import torch
import torch.nn as nn
import pandas as pd
import random
import numpy as np
import evaluate
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from torch.utils.data import DataLoader

## Cleaner draft V2

In [12]:
def import_conll_data(path:str):
    """
    Import conll_data and convert it to dataset format
    """
    with open(path, "r", encoding = "utf-8") as file:
        conll_data = file.read()

    sentences = conll_data.strip().split('\n\n')

    # Initialize empty lists to store text and labels
    text_list = []
    labels_list = []

    # Process each sentence
    for sentence in sentences:
        tokens = sentence.split('\n')
        text = " ".join(token.split()[0] for token in tokens)
        labels = " ".join(token.split()[-1] for token in tokens)
        text_list.append(text)
        labels_list.append(labels)

    data = { "text" : text_list,
            "labels" : labels_list}

    return data

data = import_conll_data(r"../../data/raw/data449.conll")

In [24]:
def import_conll_data(path: str):
    """
    Import conll_data and convert it to dataset format
    """
    with open(path, "r", encoding="utf-8") as file:
        conll_data = file.read()

    sentences = conll_data.strip().split('\n\n')

    # Initialize empty lists to store text and labels
    text_list = []
    labels_list = []

    # Process each sentence
    for sentence in sentences:
        tokens = sentence.split('\n')
        text_tokens = [token.split()[0] for token in tokens]
        label_tokens = [token.split()[-1] for token in tokens]

        text_list.append(text_tokens)
        labels_list.append(label_tokens)

    data = {"text": text_list, "labels": labels_list}

    return data
data = import_conll_data(r"../../data/raw/data449.conll")

In [35]:
print("text:", data["text"][0], "\nle type de donnée est:", type(data["text"][0]))
print("labels:", data["labels"][0], "\nle type de donnée est:", type(data["labels"][0]))

text: ['-DOCSTART-', 'evolution', 'des', 'salaires', 'de', 'base', ':', 'enveloppe', 'budgétaire', ':', 'il', 'est', 'convenu', 'entre', 'les', 'parties', 'que', 'l’enveloppe', 'budgétaire', 'consacrée', 'à', 'l’évolution', 'des', 'salaires', 'de', 'base', 'des', 'collaborateurs', 'répondant', 'aux', 'attentes', 'du', 'poste', ',', 'et', 'remplissant', 'par', 'ailleurs', 'les', 'autres', 'conditions', 'habituelles', 'd’éligibilité', 'à', 'une', 'augmentation', ',', 'représentera', '5,8', '%', 'de', 'la', 'masse', 'salariale.', 'de', 'fait', ',', 'les', 'primes', 'exprimées', 'en', 'pourcentage', 'du', 'salaire', 'de', 'base', 'augmenteront', 'ainsi', 'en', 'même', 'temps', 'que', 'le', 'salaire', 'des', 'collaborateurs', 'concernés.', 'pour', 'rappel', ',', 'conformément', 'à', 'la', 'politique', 'salariale', 'd’eli', 'lilly', '&', 'cie', ',', 'le', 'superviseur', 'prend', 'une', 'décision', 'quant', 'à', 'l’évolution', 'du', 'salaire', 'de', 'base', 'de', 'son', 'collaborateur', 'dans

In [22]:
dataset = Dataset.from_dict(data)
# dataset.features["labels"]
print(dataset['text'][0],dataset['labels'][0])


-DOCSTART- evolution des salaires de base : enveloppe budgétaire : il est convenu entre les parties que l’enveloppe budgétaire consacrée à l’évolution des salaires de base des collaborateurs répondant aux attentes du poste , et remplissant par ailleurs les autres conditions habituelles d’éligibilité à une augmentation , représentera 5,8 % de la masse salariale. de fait , les primes exprimées en pourcentage du salaire de base augmenteront ainsi en même temps que le salaire des collaborateurs concernés. pour rappel , conformément à la politique salariale d’eli lilly & cie , le superviseur prend une décision quant à l’évolution du salaire de base de son collaborateur dans son échelle , en prenant en considération : sa performance sur l’année sa performance dans la durée* la performance des collègues situés au même niveau de poste le rapport entre sa position dans l’échelle de salaire et sa performance dans la durée ( *à partir de 2023 , un collaborateur ne répondant pas aux attentes ( non

## Cleaner draft V1

In [ ]:
import torch
import pandas as pd
import random
import numpy as np
import evaluate
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from torch.utils.data import DataLoader

In [ ]:
# Load the tokenizer and model
model_checkpoint = "Jean-Baptiste/camembert-ner"
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner") #this last argument might be a mistake

In [ ]:
# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# This will print the device (either 'cuda' or 'cpu')
print("Using device:", device)

# You can then move your model and data to this device like this:
model.to(device)

### Data preparation

In [ ]:
def charge_conll(path:str):
    # Initialize a list to store the data
    conll_data = []

    # Open the CoNLL file in read mode
    with open(path, 'r', encoding='utf-8') as file:
        conll_data = file.read()
        # Read each line in the file

    # Split the data into sentences
    sentences = conll_data.strip().split('\n\n')

    # Initialize empty lists to store text and labels
    text_list = []
    labels_list = []

    # Process each sentence
    for sentence in sentences:
        tokens = sentence.split('\n')
        text = " ".join(token.split()[0] for token in tokens)
        labels = " ".join(token.split()[-1] for token in tokens)
        text_list.append(text)
        labels_list.append(labels)

    # Create a dataframe
    df = pd.DataFrame({'text': text_list, 'label': labels_list})

    return df

#we divide the data into train, test, and validation sets

def split_data(data_dict, train_size=0.8, test_size=0.1, val_size=0.1, random_seed=None):
    # Set a random seed for reproducibility
    if random_seed is not None:
        random.seed(random_seed)
        np.random.seed(random_seed)

    # Combine input_ids, attention_mask, and aligned_labels into a single list
    combined_data = list(zip(data_dict['input_ids'], data_dict['attention_mask'], data_dict['labels']))

    # Shuffle the data
    random.shuffle(combined_data)

    # Calculate the sizes of each set
    total_size = len(combined_data)
    train_size = int(train_size * total_size)
    test_size = int(test_size * total_size)
    val_size = int(val_size * total_size)

    # Split the data into train, test, and val sets
    train_data = combined_data[:train_size]
    test_data = combined_data[train_size:train_size + test_size]
    val_data = combined_data[train_size + test_size:train_size + test_size + val_size]

    # Unzip the data to restore the original structure
    train_input_ids, train_attention_mask, train_aligned_labels = zip(*train_data)
    test_input_ids, test_attention_mask, test_aligned_labels = zip(*test_data)
    val_input_ids, val_attention_mask, val_aligned_labels = zip(*val_data)

    # Create dictionaries for the train, test, and val sets
    train_set = {
        'input_ids': list(train_input_ids),
        'attention_mask': list(train_attention_mask),
        'labels': list(train_aligned_labels)
    }
    test_set = {
        'input_ids': list(test_input_ids),
        'attention_mask': list(test_attention_mask),
        'labels': list(test_aligned_labels)
    }
    val_set = {
        'input_ids': list(val_input_ids),
        'attention_mask': list(val_attention_mask),
        'labels': list(val_aligned_labels)
    }

    return train_set, test_set, val_set

def turn_sentence_to_list(sentence):
    """
    Turn a sentence into a list of words

    Args:
        sentence (str): sentence to tokenize

    Returns:
        list: list of words
    """
    return sentence.split()

def format_text(df, column):
    formated_text = []

    text = df[column]
    for i in text:
        i = turn_sentence_to_list(i)
        formated_text.append(i)

    # we add it to the dataframe
    df["formated_text"] = formated_text
    return df

def format_labels(df, column):
    labels = df[column].tolist()

    formated_labels = []

    for label in labels:
        # label = list_to_string(label)
        label = turn_sentence_to_list(label)
        formated_labels.append(label)
        # print(label)

    # we add it to the dataframe

    df["formated_labels"] = formated_labels
    return df

def change_ids(list_ids:list, new_id:dict):
    """
    change the ids of a list of ids to a new id

    Args:
        list_ids (list): list of ids
        new_id (dict): dictionary with the new id
    """
    return [new_id.get(id) for id in list_ids]

def align_labels(word_ids:list, tag_list:list):
    """
    Align the labels with the words

    Args:
        word_ids (list): list of ids of the words
        tag_list (list): list of tags
    """
    aligned_labels = []
    for i in word_ids:
        if i is None:
            aligned_labels.append(-100)
        else:
            aligned_labels.append(tag_list[i])
    return aligned_labels

def check_labels(list_labels:list):
    for label in list_labels:
        if type(label)!=int:
            print(label)
            return False
    return True

def count_error(df, column:str):
    """
    Count the number of errors - a cell containing a non numerical value - in a column of a dataframe

    Args:
        dataframe (pd.DataFrame): dataframe to check
        column (str): column to check

    Returns:
        int: number of errors
    """
    return len(df[column])- sum(df[column].apply(lambda x: check_labels(x)))

def create_iids_am(df):
    """
    Create the input ids and attention mask columns
    
    Args:
        df (pd.DataFrame): dataframe containing the data

    Returns:
        pd.DataFrame: dataframe containing the data with the input ids and attention mask columns
    """
    input_ids = []
    attention_mask = []

    for _,i in df.iterrows():
        input = i["ids"]["input_ids"]
        attention = i["ids"]["attention_mask"]

        input_ids.append(input)
        attention_mask.append(attention)

    df["input_ids"] = input_ids
    df["attention_mask"] = attention_mask
    return df

def select_columns(df, columns:list):
    """
    Select columns from a dataframe

    Args:
        df (pd.DataFrame): dataframe
        columns (list): list of columns to select

    """
    df = df[columns].copy()
    return df

def tokenize_and_align_data(path_conll:str, new_id:dict):
    """
    Tokenize the text and align the labels

    Args:
        path_conll (str): path to the conll file
        new_id (dict): dictionary with the new id

    Returns:
        pd.DataFrame: dataframe containing the data
    """
    #charge and create the df
    df = charge_conll(path_conll)
    # df["formated_text"] = df.apply(lambda x: turn_sentence_to_list(x["text"]), axis=1 )
    # df["formated_labels"] = df.apply(lambda x: turn_sentence_to_list(x["label"]), axis=1)
    df = format_text(df, "text")
    df = format_labels(df, "label")
    df["new_labels"]=df["label"].apply(lambda x: change_ids(x.split(" "), new_id))

    #tokenize and align the text
    df["ids"] = df["formated_text"].apply(lambda x: tokenizer(x, truncation=True, is_split_into_words=True))
    df = create_iids_am(df)
    df["word_ids"] = df["formated_text"].apply(lambda x: tokenizer(x, truncation=True, is_split_into_words=True).word_ids())
    df["aligned_labels"] = df.apply(lambda x: align_labels(x["word_ids"], x["new_labels"]), axis=1)
    # print(df)
    return df

def apply_tokenization(conll_path:str, new_id:dict, columns:list, new_names:dict, save_path=False, output_save_csv=None):
    """
    This function apply the tokenization and alignment to a conll file and select the columns we want to keep

    Args:
        conll_path (str): path to the conll file
        new_id (dict): dictionary with the new id
        columns (list): list of columns to keep
        new_names (dict): dictionary with the new names of the columns
        save_path (bool, optional): if True, save the dataframe in a csv file. Defaults to False.
        output_save_csv ([type], optional): path to the csv file. Defaults to None.

    Returns:
        pd.DataFrame: dataframe containing the data
    """
    df = tokenize_and_align_data(conll_path, new_id=new_id)
    df = select_columns(df, columns)
    df = df.rename(columns=new_names)
    #we check the length of the two columns so that they are of the same dimensions
    print(f"nombre de 'aligned_labels' faux: {count_error(df, 'aligned_labels')}")
    if save_path:
        df.to_csv(output_save_csv, index=False)
    return df

def final_formating(df, start:int, end:int):
    """
    Collect the data from a dataframe in a list for a given range of sentences and store them into a dictionnary

    Args:
        df (pd.DataFrame): dataframe containing the data
        start (int): index of the first sentence to collect
        end (int): index of the last sentence to collect

    Returns:
        dict: dictionnary containing the data
    """
    input_ids = []
    attention_mask = []
    labels = []

    for i in range(start, end):
        input_ids.append(df["input_ids"][i])
        attention_mask.append(df["attention_mask"][i])
        labels.append(df["aligned_labels"][i])

    data = {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

    return data

def download_and_treat_data(conll_path:str, new_id:dict, columns:list, new_names:dict, start:int, end:int, csv=False, output_save=None):
    """
    This function compile all the previous one and treat automatically all data provided as conll format.\n
    The objective is to provide a dictionnary containing the data for a given range of sentences with only one function. \n
    The same results can be obtained by parts with apply_tokenization and final_formating functions. This one is more a convenience function.\n
    It is also possible to obtain a csv file if csv is set to True and a path is provided in output_save.

    Args:
        conll_path (str): path to the conll file
        new_id (dict): dictionnary to change the labels
        columns (list): columns to keep
        new_names (dict): new names of the columns
        start (int): index of the first sentence to collect
        end (int): index of the last sentence to collect
    
    Returns:
        dict: dictionnary containing the data
    """
    df = apply_tokenization(conll_path, new_id=new_id, columns=columns, new_names=new_names)

    if csv:
        df.to_csv(output_save, index=False)

    data = final_formating(df, start, end)
    return data, df

In [ ]:
new_id = {"O":0,"B-SYND":1, "I-SYND":1, "SYND":1, "B-DIR":2, "I-DIR":2, "DIR":2, "B-DATE":4, "I-DATE":4, "DATE":4, "B-ENT":3, "I-ENT":3, "ENT":3, "B-CAD":5, "I-CAD":5, "CAD":5, "B-INT":6, "I-INT":6, "INT":6,"B-OUV":7, "I-OUV":7, "OUV":7, "B-NCAD":8, "I-NCAD":8, "NCAD":8,"B-NOUV":9, "I-NOUV":9, "NOUV":9, "B-TOUS":10,"I-TOUS":10, "TOUS":10, "B-AG CAD":11,"I-AG CAD":11, "AG CAD":11, "B-AG INT":12, "I-AG INT":12, "AG INT":12, "B-AG OUV":13, "I-AG OUV":13, "AG OUV":13,"B-AG NCAD":14, "I-AG NCAD":14, "AG NCAD":14, "B-AG NOUV":15, "I-AG NOUV":15, "AG NOUV":15,"B-AI CAD":16, "I-AI CAD":16,"AI CAD":16,"B-AI INT":17, "I-AI INT":17, "AI INT":17,"B-AI OUV":18, "I-AI OUV":18,"AI OUV":18,"B-AI NCAD":19, "I-AI NCAD":19,"AI NCAD":19,"B-AI NOUV":20, "I-AI NOUV":20, "AI NOUV":20, "B-AG":21, "I-AG":21, "AG":21,"B-AI":22, "I-AI":22,"AI":22,"B-ATOT":23,"I-ATOT":23, "ATOT":23,"B-ATOT CAD":24, "I-ATOT CAD":24, "ATOT CAD":24,"B-ATOT INT":25, "I-ATOT INT":25,"ATOT INT":25,"B-ATOT OUV":26, "I-ATOT OUV":26, "ATOT OUV":26,"B-ATOT NCAD":27, "I-ATOT NCAD":27, "ATOT NCAD":27,"B-ATOT NOUV":28, "I-ATOT NOUV":28, "ATOT NOUV":28,"B-PPV":29, "I-PPV":29, "PPV":29,"B-PPVm":30, "I-PPVm":30, "PPVm":30}
columns = ["formated_text", "formated_labels","new_labels","word_ids", "input_ids", "attention_mask", "aligned_labels"]
new_names = {"formated_text": "text", "new_labels": "label", "word_ids": "word_ids", "aligned_labels": "aligned_labels"}
conll_path = r"../../data/raw/data449.conll"
start = 0
end = 449
output_save = r"../../data/intermediate/data449_token_V2.csv"

data, df_final = download_and_treat_data(conll_path=conll_path, new_id=new_id, columns=columns, new_names=new_names, start=start, end=end, csv=True, output_save=output_save)

train_data, test_data, val_data = split_data(data)

# Create a Dataset object
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)
val_dataset = Dataset.from_dict(val_data)

# Create a list of labels
reverse_id = {v: k for k, v in new_id.items()}
second_elements = [value for value in reverse_id.values()]
label_list = second_elements

# Load the tokenizer and model
model_checkpoint = "Jean-Baptiste/camembert-ner"
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner", num_labels=len(label_list), ignore_mismatched_sizes=True) #this last argument might be a mistake

# import data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors='pt')

# Dataloader
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=data_collator,
    batch_size=8,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
train_dataset

### Training Arguments

In [ ]:
batch_size = 8

# training arguments
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    # push_to_hub=True,
)



# metric used for evaluation
metric = evaluate.load("seqeval")

# We want to do a metrics function to compute the accuracy of the model

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

### Train

In [ ]:
trainer.train()
print(p)

## try with conll

In [ ]:
def charge_conll(path:str):
    # Initialize a list to store the data
    conll_data = []

    # Open the CoNLL file in read mode
    with open(path, 'r', encoding='utf-8') as file:
        conll_data = file.read()
        # Read each line in the file

    # Split the data into sentences
    sentences = conll_data.strip().split('\n\n')

    # Initialize empty lists to store text and labels
    text_list = []
    labels_list = []

    # Process each sentence
    for sentence in sentences:
        tokens = sentence.split('\n')
        text = " ".join(token.split()[0] for token in tokens)
        labels = " ".join(token.split()[-1] for token in tokens)
        text_list.append(text)
        labels_list.append(labels)

    # Create a dataframe
    df = pd.DataFrame({'text': text_list, 'label': labels_list})

    return df

In [ ]:
# Specify the file path to your CoNLL file
conll_file_path = r'../../data/raw/data449.conll'  # Replace with your file path

# Initialize a list to store the data
conll_data = []

# Open the CoNLL file in read mode
with open(conll_file_path, 'r', encoding='utf-8') as file:
    conll_data = file.read()
    # Read each line in the file

# Split the data into sentences
sentences = conll_data.strip().split('\n\n')

# Initialize empty lists to store text and labels
text_list = []
labels_list = []

# Process each sentence
for sentence in sentences:
    tokens = sentence.split('\n')
    text = " ".join(token.split()[0] for token in tokens)
    labels = " ".join(token.split()[-1] for token in tokens)
    text_list.append(text)
    labels_list.append(labels)

# Create a dataframe
df = pd.DataFrame({'text': text_list, 'label': labels_list})

# Print the dataframe
print(df)

In [ ]:
def turn_sentence_to_list(sentence):
    """
    Turn a sentence into a list of tokens

    Args:
        sentence (str): sentence to tokenize

    Returns:
        list: list of tokens
    """
    return [token for token in sentence.split(" ")]

# def turn_sentence_to_list(sentence):
#     """
#     Turn a sentence into a list of words

#     Args:
#         sentence (str): sentence to tokenize

#     Returns:
#         list: list of words
#     """
#     return sentence.split()


def list_to_string(input_list, separator=' '):
    """
    Convert a list of strings into a single string with elements separated by a specified separator.

    :param input_list: List of strings to be converted.
    :param separator: The separator to use between elements (default is a space).
    :return: A single string containing the list elements.
    """
    return separator.join(input_list)

def change_ids(list_ids:list, new_id:dict):
    return [new_id.get(id) for id in list_ids]

In [ ]:
new_id = {"O":0,"B-SYND":1, "I-SYND":1, "SYND":1, "B-DIR":2, "I-DIR":2, "DIR":2, "B-DATE":4, "I-DATE":4, "DATE":4, "B-ENT":3, "I-ENT":3, "ENT":3, "B-CAD":5, "I-CAD":5, "CAD":5, "B-INT":6, "I-INT":6, "INT":6,"B-OUV":7, "I-OUV":7, "OUV":7, "B-NCAD":8, "I-NCAD":8, "NCAD":8,"B-NOUV":9, "I-NOUV":9, "NOUV":9, "B-TOUS":10,"I-TOUS":10, "TOUS":10, "B-AG CAD":11,"I-AG CAD":11, "AG CAD":11, "B-AG INT":12, "I-AG INT":12, "AG INT":12, "B-AG OUV":13, "I-AG OUV":13, "AG OUV":13,"B-AG NCAD":14, "I-AG NCAD":14, "AG NCAD":14, "B-AG NOUV":15, "I-AG NOUV":15, "AG NOUV":15,"B-AI CAD":16, "I-AI CAD":16,"AI CAD":16,"B-AI INT":17, "I-AI INT":17, "AI INT":17,"B-AI OUV":18, "I-AI OUV":18,"AI OUV":18,"B-AI NCAD":19, "I-AI NCAD":19,"AI NCAD":19,"B-AI NOUV":20, "I-AI NOUV":20, "AI NOUV":20, "B-AG":21, "I-AG":21, "AG":21,"B-AI":22, "I-AI":22,"AI":22,"B-ATOT":23,"I-ATOT":23, "ATOT":23,"B-ATOT CAD":24, "I-ATOT CAD":24, "ATOT CAD":24,"B-ATOT INT":25, "I-ATOT INT":25,"ATOT INT":25,"B-ATOT OUV":26, "I-ATOT OUV":26, "ATOT OUV":26,"B-ATOT NCAD":27, "I-ATOT NCAD":27, "ATOT NCAD":27,"B-ATOT NOUV":28, "I-ATOT NOUV":28, "ATOT NOUV":28,"B-PPV":29, "I-PPV":29, "PPV":29,"B-PPVm":30, "I-PPVm":30, "PPVm":30}

df["new_labels"]=df["label"].apply(lambda x: change_ids(x.split(" "), new_id))

In [ ]:
print(df["new_labels"][5])
print(df["label"][5])

In [ ]:
# we check that we haven't left non-numeric labels


def check_labels(list_labels:list):
    for label in list_labels:
        if type(label)!=int:
            print(label)
            return False
    return True

# amount = [df["new_labels"].apply(lambda x: check_labels(x))]

def count_error(dataframe, column:str):
    """
    Count the number of errors - a cell containing a non numerical value - in a column of a dataframe

    Args:
        dataframe (pd.DataFrame): dataframe to check
        column (str): column to check

    Returns:
        int: number of errors
    """
    return len(df[column])- sum(dataframe[column].apply(lambda x: check_labels(x)))

# print(f"nombre d'erreurs: {count_error(df, 'new_labels')}")


In [ ]:
texts = df["text"].tolist()
start = 0

for text in texts: 
    text = turn_sentence_to_list(text)
    tokens = tokenizer(text, is_split_into_words=True)
    tokens_mod = tokenizer.convert_ids_to_tokens(tokens["input_ids"])
    length = len(tokens_mod)
    if length > 512:
        start += 1
        print(length)
    else:
        pass

print(start)

JB NER is probably not suited since we lost 182 texts, which is too much, due to context limit. We can eventually try to split the text into 512 tokens chunks. 

In [ ]:
import pandas as pd
from collections import defaultdict

# Sample data in CoNLL format
conll_data = r'../../data/raw/data449.conll'

with open(conll_data, 'r', encoding='utf-8') as file:
    conll_data = file.read()

# Split the data into sentences
sentences = conll_data.strip().split('\n\n')

# Initialize empty lists and a variable to keep track of the current sentence length
text_list = []
labels_list = []
current_length = 0

# Define the maximum token length for a chunk
max_chunk_length = 512

# Initialize a dictionary to keep track of chunks
chunked_data = defaultdict(list)

# Process each sentence
for sentence in sentences:
    tokens = sentence.split('\n')
    text = " ".join(token.split()[0] for token in tokens)
    labels = " ".join(token.split()[-1] for token in tokens)
    
    # Check if adding this sentence will exceed the maximum chunk length
    if current_length + len(tokens) <= max_chunk_length:
        # Add this sentence to the current chunk
        text_list.append(text)
        labels_list.append(labels)
        current_length += len(tokens)
    else:
        # Start a new chunk
        chunked_data['text'].append(text_list)
        chunked_data['label'].append(labels_list)
        
        # Reset the current chunk
        text_list = [text]
        labels_list = [labels]
        current_length = len(tokens)

# Add the last chunk
chunked_data['text'].append(text_list)
chunked_data['label'].append(labels_list)

# Create a dataframe from the chunked data
df = pd.DataFrame(chunked_data)

# Print the dataframe
print(df)


In [ ]:
# Example usage:
def format_text(df):
    formated_text = []

    text = df["text"]
    for i in text:
        i = turn_sentence_to_list(i)
        formated_text.append(i)

    # we add it to the dataframe
    df["formated_text"] = formated_text
    return df

# formated_text = []

# text = df["text"]
# for i in text:
#     i = turn_sentence_to_list(i)
#     formated_text.append(i)

# # we add it to the dataframe
# df["formated_text"] = formated_text
# df = format_text(df)


now we format the label to the right format

In [ ]:
def format_labels(df):
    labels = df["label"].tolist()

    formated_labels = []

    for label in labels:
        # label = list_to_string(label)
        label = turn_sentence_to_list(label)
        formated_labels.append(label)
        # print(label)

    # we add it to the dataframe

    df["formated_labels"] = formated_labels
    return df

In [ ]:
def select_columns(df, columns:list):
    df = df[columns].copy()
    return df

# df = select_columns(df, columns=["formated_text", "formated_labels"])

# df = df.rename(columns={"formated_text": "text", "formated_labels": "label"})



In [ ]:
# We check the length of the two columns so that they are of the same dimensions

start = 0

for i in range(len(df["text"])):
    if len(df['text'][i]) != len(df['label'][i]):
        start += 1
    else:
        pass

print("les éléments dont la dimension diverge sont au nombre de",start)

In [ ]:
df.head()

We start tokenizing the text

In [ ]:
type(df["text"][0][0])

In [ ]:
tokenized_input = tokenizer(df["text"][0], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokenized_input)

In [ ]:
token_input = tokenizer(df['text'][0], is_split_into_words=True)
print(token_input)

# tokens = tokenizer.convert_ids_to_tokens(token_input["input_ids"])
# print(tokens)

In [ ]:
# word_ids = token_input.word_ids()
# print(word_ids)

def align_labels(word_ids:list, tag_list:list):
    aligned_labels = []
    for i in word_ids:
        if i is None:
            aligned_labels.append(-100)
        else:
            aligned_labels.append(tag_list[i])
    return aligned_labels

# aligned_labels = align_labels(word_ids, tag_list=df["new_labels"][0])
# aligned_labels = create_aligned_labels(word_ids, example=df)

# print(aligned_labels)
# print(len(aligned_labels))
# print(len(word_ids))

In [ ]:
df["word_ids"] = df["text"].apply(lambda x: tokenizer(x, is_split_into_words=True).word_ids())
df["aligned_labels"] = df.apply(lambda x: align_labels(x["word_ids"], x["label"]), axis=1)

In [ ]:
list(zip(tokens, aligned_labels))

In [ ]:
def create_iids_am(df):
    input_ids = []
    attention_mask = []

    for _,i in df.iterrows():
        input = i["ids"]["input_ids"]
        attention = i["ids"]["attention_mask"]

        input_ids.append(input)
        attention_mask.append(attention)

    df["input_ids"] = input_ids
    df["attention_mask"] = attention_mask
    return df

# df = create_iids_am(df)
# df[["input_ids", "attention_mask"]].head()
# df_ids = df[["ids"]].copy()
# df_ids["ids"][0]

In [ ]:
new_id = {"O":0,"B-SYND":1, "I-SYND":1, "SYND":1, "B-DIR":2, "I-DIR":2, "DIR":2, "B-DATE":4, "I-DATE":4, "DATE":4, "B-ENT":3, "I-ENT":3, "ENT":3, "B-CAD":5, "I-CAD":5, "CAD":5, "B-INT":6, "I-INT":6, "INT":6,"B-OUV":7, "I-OUV":7, "OUV":7, "B-NCAD":8, "I-NCAD":8, "NCAD":8,"B-NOUV":9, "I-NOUV":9, "NOUV":9, "B-TOUS":10,"I-TOUS":10, "TOUS":10, "B-AG CAD":11,"I-AG CAD":11, "AG CAD":11, "B-AG INT":12, "I-AG INT":12, "AG INT":12, "B-AG OUV":13, "I-AG OUV":13, "AG OUV":13,"B-AG NCAD":14, "I-AG NCAD":14, "AG NCAD":14, "B-AG NOUV":15, "I-AG NOUV":15, "AG NOUV":15,"B-AI CAD":16, "I-AI CAD":16,"AI CAD":16,"B-AI INT":17, "I-AI INT":17, "AI INT":17,"B-AI OUV":18, "I-AI OUV":18,"AI OUV":18,"B-AI NCAD":19, "I-AI NCAD":19,"AI NCAD":19,"B-AI NOUV":20, "I-AI NOUV":20, "AI NOUV":20, "B-AG":21, "I-AG":21, "AG":21,"B-AI":22, "I-AI":22,"AI":22,"B-ATOT":23,"I-ATOT":23, "ATOT":23,"B-ATOT CAD":24, "I-ATOT CAD":24, "ATOT CAD":24,"B-ATOT INT":25, "I-ATOT INT":25,"ATOT INT":25,"B-ATOT OUV":26, "I-ATOT OUV":26, "ATOT OUV":26,"B-ATOT NCAD":27, "I-ATOT NCAD":27, "ATOT NCAD":27,"B-ATOT NOUV":28, "I-ATOT NOUV":28, "ATOT NOUV":28,"B-PPV":29, "I-PPV":29, "PPV":29,"B-PPVm":30, "I-PPVm":30, "PPVm":30}
columns = ["formated_text", "new_labels","word_ids", "input_ids", "attention_mask", "aligned_labels"]
new_names = {"formated_text": "text", "new_labels": "label", "word_ids": "word_ids", "aligned_labels": "aligned_labels"}

def tokenize_and_align_data(path_conll:str, new_id:dict):
    #charge and create the df
    df = charge_conll(path_conll)
    df = format_text(df)
    df = format_labels(df)
    df["new_labels"]=df["label"].apply(lambda x: change_ids(x.split(" "), new_id))

    #tokenize and align the text
    df["ids"] = df["formated_text"].apply(lambda x: tokenizer(x, truncation=True, is_split_into_words=True))
    df = create_iids_am(df)
    df["word_ids"] = df["formated_text"].apply(lambda x: tokenizer(x, truncation=True, is_split_into_words=True).word_ids())
    df["aligned_labels"] = df.apply(lambda x: align_labels(x["word_ids"], x["new_labels"]), axis=1)
    return df

def apply_tokenization(conll_path:str, new_id:dict, columns:list, new_names:dict, save_path=False, output_save=None):
    df = tokenize_and_align_data(conll_path, new_id=new_id)
    df = select_columns(df, columns)
    df = df.rename(columns=new_names)
    #we check the length of the two columns so that they are of the same dimensions
    # print(f"nombre de 'aligned_labels' faux: {count_error(df, 'aligned_labels')}")
    if save_path:
        df.to_csv(output_save, index=False)
    return df

# df = apply_tokenization(r"..\..\data\raw\data449.conll", new_id=new_id, columns=columns, new_names=new_names, save_path=True, output_save=r"../../data/intermediate/data499_token.csv")
# df.head()
# df = tokenize_and_align_data(r"..\..\data\raw\data449.conll", new_id=new_id)
# df = select_columns(df, columns)
# df = df.rename(columns=new_names)
# #we check the length of the two columns so that they are of the same dimensions
# print(f"nombre de 'aligned_labels' faux: {count_error(df, 'aligned_labels')}")
# # df.to_csv(r"../../data/intermediate/data499_token.csv", index=False)
# df

In [ ]:
list_text = df["text"].tolist()

for i in list_text:
    if len(i) > 512:
        print(len(i))
    else:
        print("ok")

In [ ]:
# the objective is to collect all the inputs_ids, attention_mask and labels in a list for a given range of sentences and store them into a dictionnary

def final_formating(df, start:int, end:int):
    """
    Collect the data from a dataframe in a list for a given range of sentences and store them into a dictionnary

    Args:
        df (pd.DataFrame): dataframe containing the data
        start (int): index of the first sentence to collect
        end (int): index of the last sentence to collect

    Returns:
        list: list of the data
    """
    input_ids = []
    attention_mask = []
    labels = []

    for i in range(start, end):
        input_ids.append(df["input_ids"][i])
        attention_mask.append(df["attention_mask"][i])
        labels.append(df["aligned_labels"][i])

    data = {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

    return data
    
# data = final_formating(df, 0, 5)
# print(data)
# for _,i in df.iterrows():
    

In [ ]:
def download_and_treat_data(conll_path:str, new_id:dict, columns:list, new_names:dict, start:int, end:int, csv=False, output_save=None):
    """
    This function compile all the previous one and treat automatically all data provided as conll format.\n
    The objective is to provide a dictionnary containing the data for a given range of sentences with only one function. \n
    The same results can be obtained by parts with apply_tokenization and final_formating functions. This one is more a convenience function.\n
    It is also possible to obtain a csv file if csv is set to True and a path is provided in output_save.

    Args:
        conll_path (str): path to the conll file
        new_id (dict): dictionnary to change the labels
        columns (list): columns to keep
        new_names (dict): new names of the columns
        start (int): index of the first sentence to collect
        end (int): index of the last sentence to collect
    
    Returns:
        dict: dictionnary containing the data
    """
    df = apply_tokenization(conll_path, new_id=new_id, columns=columns, new_names=new_names)

    if csv:
        df.to_csv(output_save, index=False)

    data = final_formating(df, start, end)
    return data

In [ ]:
new_id = {"O":0,"B-SYND":1, "I-SYND":1, "SYND":1, "B-DIR":2, "I-DIR":2, "DIR":2, "B-DATE":4, "I-DATE":4, "DATE":4, "B-ENT":3, "I-ENT":3, "ENT":3, "B-CAD":5, "I-CAD":5, "CAD":5, "B-INT":6, "I-INT":6, "INT":6,"B-OUV":7, "I-OUV":7, "OUV":7, "B-NCAD":8, "I-NCAD":8, "NCAD":8,"B-NOUV":9, "I-NOUV":9, "NOUV":9, "B-TOUS":10,"I-TOUS":10, "TOUS":10, "B-AG CAD":11,"I-AG CAD":11, "AG CAD":11, "B-AG INT":12, "I-AG INT":12, "AG INT":12, "B-AG OUV":13, "I-AG OUV":13, "AG OUV":13,"B-AG NCAD":14, "I-AG NCAD":14, "AG NCAD":14, "B-AG NOUV":15, "I-AG NOUV":15, "AG NOUV":15,"B-AI CAD":16, "I-AI CAD":16,"AI CAD":16,"B-AI INT":17, "I-AI INT":17, "AI INT":17,"B-AI OUV":18, "I-AI OUV":18,"AI OUV":18,"B-AI NCAD":19, "I-AI NCAD":19,"AI NCAD":19,"B-AI NOUV":20, "I-AI NOUV":20, "AI NOUV":20, "B-AG":21, "I-AG":21, "AG":21,"B-AI":22, "I-AI":22,"AI":22,"B-ATOT":23,"I-ATOT":23, "ATOT":23,"B-ATOT CAD":24, "I-ATOT CAD":24, "ATOT CAD":24,"B-ATOT INT":25, "I-ATOT INT":25,"ATOT INT":25,"B-ATOT OUV":26, "I-ATOT OUV":26, "ATOT OUV":26,"B-ATOT NCAD":27, "I-ATOT NCAD":27, "ATOT NCAD":27,"B-ATOT NOUV":28, "I-ATOT NOUV":28, "ATOT NOUV":28,"B-PPV":29, "I-PPV":29, "PPV":29,"B-PPVm":30, "I-PPVm":30, "PPVm":30}
columns = ["formated_text", "formated_labels","new_labels","word_ids", "input_ids", "attention_mask", "aligned_labels"]
new_names = {"formated_text": "text", "new_labels": "label", "word_ids": "word_ids", "aligned_labels": "aligned_labels"}
conll_path = r"..\..\data\raw\data449.conll"
start = 0
end = 449

data = download_and_treat_data(conll_path=conll_path, new_id=new_id, columns=columns, new_names=new_names, start=start, end=end, csv=True, output_save=r"../../data/intermediate/data449_token.csv")

In [ ]:
#we divide the data into train, test, and validation sets

def split_data(data_dict, train_size=0.8, test_size=0.1, val_size=0.1, random_seed=None):
    # Set a random seed for reproducibility
    if random_seed is not None:
        random.seed(random_seed)
        np.random.seed(random_seed)

    # Combine input_ids, attention_mask, and aligned_labels into a single list
    combined_data = list(zip(data_dict['input_ids'], data_dict['attention_mask'], data_dict['labels']))

    # Shuffle the data
    random.shuffle(combined_data)

    # Calculate the sizes of each set
    total_size = len(combined_data)
    train_size = int(train_size * total_size)
    test_size = int(test_size * total_size)
    val_size = int(val_size * total_size)

    # Split the data into train, test, and val sets
    train_data = combined_data[:train_size]
    test_data = combined_data[train_size:train_size + test_size]
    val_data = combined_data[train_size + test_size:train_size + test_size + val_size]

    # Unzip the data to restore the original structure
    train_input_ids, train_attention_mask, train_aligned_labels = zip(*train_data)
    test_input_ids, test_attention_mask, test_aligned_labels = zip(*test_data)
    val_input_ids, val_attention_mask, val_aligned_labels = zip(*val_data)

    # Create dictionaries for the train, test, and val sets
    train_set = {
        'input_ids': list(train_input_ids),
        'attention_mask': list(train_attention_mask),
        'labels': list(train_aligned_labels)
    }
    test_set = {
        'input_ids': list(test_input_ids),
        'attention_mask': list(test_attention_mask),
        'labels': list(test_aligned_labels)
    }
    val_set = {
        'input_ids': list(val_input_ids),
        'attention_mask': list(val_attention_mask),
        'labels': list(val_aligned_labels)
    }

    return train_set, test_set, val_set

In [ ]:
train_data, test_data, val_data = split_data(data)

we get rid of the list with a dimension superior to 512 as the model doesn't hold more

Now we turn into tensors our data so that BERT can read them

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)
val_dataset = Dataset.from_dict(val_data)

In [ ]:
# list of labels
reverse_id = {v: k for k, v in new_id.items()}
second_elements = [value for value in reverse_id.values()]
label_list = second_elements
print(label_list)

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "Jean-Baptiste/camembert-ner"
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner", num_labels=len(label_list), ignore_mismatched_sizes=True) #this last argument might be a mistake

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors='pt')

In [ ]:
data_collator(train_dataset)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=data_collator,
    batch_size=8,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
next(iter(train_dataloader))

In [ ]:
model(**(next(iter(train_dataloader))))

Training Argument

In [ ]:
from transformers import TrainingArguments

batch_size = 8

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    # push_to_hub=True,
)

In [ ]:
metric = load_metric("seqeval")

In [ ]:
data_token = pd.read_csv(r"../../data/intermediate/data449_token.csv")
data_token.head()

In [ ]:
def string_to_list(label_string):
    # Split the string by spaces and remove any leading/trailing spaces
    labels_list = label_string.strip().split()
    return labels_list

list_tags = data_token["formated_labels"].apply(lambda x: string_to_list(x))

print(list_tags)

labels = [label_list[i] for i in list_tags]
# metric.compute(predictions=[labels], references=[labels])

In [ ]:
# We want to do a metrics function to compute the accuracy of the model

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

#### drafty draft

In [ ]:
df_token = df[['input_ids']].copy()
print(df_token["input_ids"][0]["input_ids"])

In [ ]:
print(df["aligned_labels"][0])
df_test = df["aligned_labels"].apply(lambda x :change_ids(x, new_id=new_id))
print(df_test[0])

In [ ]:
example = df.iloc[0]

In [ ]:
tokenized_input = tokenizer(example["text"], is_split_into_words=True)
aligned_labels = [-100 if i is None else example[f"label"][i] for i in word_ids]
print(example["label"])
print(len(word_ids))
print(len(aligned_labels))

In [ ]:
list(zip(tokens_mod, aligned_labels))

## others

Load a dataset for training from Huggingface

In [ ]:
# If you like it you can download it
data = import_label_studio_data("../../data/raw/data449.json")

df = pd.DataFrame(data, columns = ['text', 'label'])

In [ ]:
df.head()

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

In [ ]:
def turn_sentence_to_list(sentence):
    """
    Turn a sentence into a list of tokens

    Args:
        sentence (str): sentence to tokenize

    Returns:
        list: list of tokens
    """
    return [token for token in sentence.split(" ")]

In [ ]:
for i in range(len(df["text"])):
    df["text"][i] = turn_sentence_to_list(df["text"][i])

df.head()

In [ ]:
length = []

for i in range(len(df["text"])):
    element = df["text"][i]
    length.append(len(element))
    # print(len(df["text"][i])) 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors

plt.hist(length, bins=20, color = "lightgreen", edgecolor='black') 

# Add labels and title
plt.xlabel('Longeur (élément)')
plt.ylabel('Fréquence')
plt.title('Histogram de la longueurs des phrases en élément')

# Show the histogram
plt.show()


In [ ]:
import pandas as pd

# Sample DataFrame
data = {
    'text': ["evolution des salaires de base : enveloppe"],
    'label': [{'entities': [(22, 31, 'ATOT')]}]
}

df = pd.DataFrame(data)

# Function to tokenize text while preserving whitespace
def tokenize_text(row):
    text = row['text']
    tokens = []
    start = 0

    for start, end, entity in row['label']['entities']:
        # Add non-entity text
        tokens.extend(text[start:end].split())
        start = end

    # Add any remaining text after the last entity
    tokens.extend(text[start:].split())
    
    return tokens

# Apply the function to create a new column
df['tokenized_text'] = df.apply(tokenize_text, axis=1)

# Display the DataFrame with tokenized text while preserving whitespace
print(df)


In [ ]:
for i in range(len(df["label"])):
    print(df["label"][i]["entities"])
    


In [ ]:
text = df['text'][0]
text = turn_sentence_to_list(text)
# print(type(text))
tokens = tokenizer(text, is_split_into_words=True)
# print(tokens)

tokens_mod = tokenizer.convert_ids_to_tokens(tokens["input_ids"])
print(tokens_mod)


In [ ]:
word_ids = tokens.word_ids()
# aligned_labels = [-100 if i is None else text["label"][i] for i in word_ids]
for i in word_ids:
    # print(i)
    if i is None:
        print(i)
    else:
        print(text["label"][i])

In [ ]:
import torch
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)